We have created a search & content-based recommendation system using PyTerrier.

#Installing Required Packages and Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
%matplotlib inline

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
exclude = set(string.punctuation)
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anoosh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anoosh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anoosh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!pip install python-terrier

In [5]:
import pyterrier as pt
#if not pt.started():
 # pt.init()

In [6]:
!pip install -U sentence-transformers

In [8]:
!pip install neuspell
!pip install -e neuspell/
!git clone https://github.com/neuspell/neuspell; cd neuspell
import os
#os.chdir("/content/neuspell")

!pip install -r /content/neuspell/extras-requirements.txt
!python -m spacy download en_core_web_sm

#Unzipping the multi-linguistic packages
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip *.zip

  Using cached neuspell-1.0.0-py3-none-any.whl (184 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\scripts\\__init__.py'
Consider using the `--user` option or check the permissions.

ERROR: neuspell/ is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).
'git' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/neuspell/extras-requirements.txt'
C:\ProgramData\Anaconda3\python.exe: No module named spacy
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable

In [9]:
from neuspell import BertChecker
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

data folder is set to `C:\ProgramData\Anaconda3\lib\site-packages\neuspell\../data` script


#Loading the Dataset

We have used the FlipKart dataset.

In [ ]:
path="flipkart_com-ecommerce_sample.csv"

In [ ]:
df=pd.read_csv(path, na_values=["No rating available"])
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,NaN,NaN,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,NaN,NaN,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,NaN,NaN,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           1849 non-null   float64
 12  overall_rating           1849 non-null   float64
 13  brand                    14136 non-null  object 
 14  product_specifications

#Data Preprocessing

Making 'category_tree' into a simple list:

In [ ]:
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.strip('[]'))
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.strip('"'))
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.split('>>'))

Dropping unwanted columns:

In [ ]:
df.drop(['crawl_timestamp','product_url','image',"retail_price","discounted_price","is_FK_Advantage_product","product_rating","overall_rating","product_specifications"],axis=1,inplace=True)

In [ ]:
df.head()

,uniq_id,product_name,product_category_tree,pid,description,brand
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2FF9KEDEFGF,Key Features of Alisha Solid Women's Cycling S...,Alisha
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[Furniture , Living Room Furniture , Sofa Be...",SBEEH3QGU7MFYJFY,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[Footwear , Women's Footwear , Ballerinas , ...",SHOEH4GRSUBJGZXE,Key Features of AW Bellies Sandals Wedges Heel...,AW
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2F6HUZMQ6SJ,Key Features of Alisha Solid Women's Cycling S...,Alisha
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[Pet Supplies , Grooming , Skin & Coat Care ...",PSOEH3ZYDMSYARJ5,Specifications of Sicons All Purpose Arnica Do...,Sicons


Dropping duplicate products:

In [ ]:
uniq_prod=df.copy()
uniq_prod.drop_duplicates(subset ="product_name", 
                     keep = "first", inplace = True)
uniq_prod.shape

(12676, 6)

### Data Cleaning

Removing stop words & punctuations.
Performing tokenizing & lemmatization.

In [ ]:
def filter_keywords(doc):
    doc=doc.lower()
    stop_free = " ".join([i for i in doc.split() if i not in stop_words])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    word_tokens = word_tokenize(punc_free)
    filtered_sentence = [(lem.lemmatize(w, "v")) for w in word_tokens]
    return filtered_sentence

In [ ]:
uniq_prod['product'] = uniq_prod['product_name'].apply(filter_keywords)
uniq_prod['description'] = uniq_prod['description'].astype("str").apply(filter_keywords)
uniq_prod['brand'] = uniq_prod['brand'].astype("str").apply(filter_keywords)

In [ ]:
uniq_prod["keywords"]=uniq_prod['product']+uniq_prod['brand']+ df['product_category_tree']+uniq_prod['description']
uniq_prod["keywords"] = uniq_prod["keywords"].apply(lambda x: ' '.join(x))

The column 'all_meta' has all the important keywords related to the corresponding product.

In [ ]:
uniq_prod["keywords"].head()

0    alisha solid womens cycle short alisha Clothin...
1    fabhomedecor fabric double sofa bed fabhomedec...
2    aw belly aw Footwear   Women's Footwear   Ball...
4    sicons purpose arnica dog shampoo sicons Pet S...
5    eternal gandhi super series crystal paper weig...
Name: keywords, dtype: object

In [ ]:
uniq_prod['docno']=uniq_prod['product_name']

In [ ]:
uniq_prod.drop(['product','brand','pid','product_name'],axis=1,inplace=True)

#Final Dataset

In [ ]:
uniq_prod.head()

,uniq_id,product_category_tree,description,keywords,docno
0,c2d766ca982eca8304150849735ffef9,"[Clothing , Women's Clothing , Lingerie, Sle...","[key, feature, alisha, solid, womens, cycle, s...",alisha solid womens cycle short alisha Clothin...,Alisha Solid Women's Cycling Shorts
1,7f7036a6d550aaa89d34c77bd39a5e48,"[Furniture , Living Room Furniture , Sofa Be...","[fabhomedecor, fabric, double, sofa, bed, fini...",fabhomedecor fabric double sofa bed fabhomedec...,FabHomeDecor Fabric Double Sofa Bed
2,f449ec65dcbc041b6ae5e6a32717d01b,"[Footwear , Women's Footwear , Ballerinas , ...","[key, feature, aw, belly, sandals, wedge, heel...",aw belly aw Footwear Women's Footwear Ball...,AW Bellies
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,"[Pet Supplies , Grooming , Skin & Coat Care ...","[specifications, sicons, purpose, arnica, dog,...",sicons purpose arnica dog shampoo sicons Pet S...,Sicons All Purpose Arnica Dog Shampoo
5,c2a17313954882c1dba461863e98adf2,[Eternal Gandhi Super Series Crystal Paper Wei...,"[key, feature, eternal, gandhi, super, series,...",eternal gandhi super series crystal paper weig...,Eternal Gandhi Super Series Crystal Paper Weig...


#Building the Recommendation System

Indexing the data (Do not re-run):

In [ ]:
!rm -rf /content/iter_index_porter
pd_indexer = pt.DFIndexer("/content/pd_index")
indexref = pd_indexer.index(uniq_prod["keywords"], uniq_prod["docno"])

Implementing NeuSpell SpellCheck on query:

In [ ]:
spellcheck = BertChecker()
spellcheck.from_pretrained(
    ckpt_path=f"/content/multi_cased_L-12_H-768_A-12"  # "<folder where the model is saved>"
)

In [ ]:
X=input("Search Engine:")
query=spellcheck.correct(X)
print(query)

Search Engine:womns clothin
women clothing


Performing ranking and retrieval using PyTerrier and SentenceBERT:

In [ ]:
prod_ret = pt.BatchRetrieve(indexref, wmodel='TF_IDF', properties={'termpipelines': 'Stopwords'})
pr=prod_ret.compile()
output=pr.search(query)
docno=list(output['docno'])
transform=model.encode(docno)

Applying 8 rules


Creating embeddings and re-ranking using PyTerrier & Cosine Similarity:

In [ ]:
embedding={}
for i,product in enumerate(docno):
    embedding[product]=transform[i]

In [ ]:
q_embedding=model.encode(query).reshape(1,-1)
l=[]
for product in embedding.keys():
    score=cosine_similarity(q_embedding,embedding[product].reshape(1,-1))[0][0]
    l.append([product,score])

In [ ]:
output2=pd.DataFrame(l,columns=['product_name','score'])

#Final Output

In [ ]:
output2.sort_values(by='score',ascending=False).head(10)

,product_name,score
96,People Women's Dress,0.862905
235,Nasha Women's Gathered Dress,0.763813
225,Kasturi Women's Gathered Dress,0.754854
165,Sbuys Women's Gathered Dress,0.751761
119,Teemoods Women's Camisole,0.748863
71,"Viyasha Women's, Girl's Shapewear",0.747937
231,Karishma Women's Gathered Dress,0.742156
183,Kwardrobe Women's Gathered Dress,0.741498
218,Modimania Women's Gathered Dress,0.740214
742,IDK Woman Women's Shift Dress,0.736790
